In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-24 11:39:54--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-24 11:39:54 (8.42 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [58]:
import minsearch
import os

In [59]:
import json

In [60]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [61]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [62]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [63]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"])
#Esto hace como una query tipo select

In [64]:
query = "the course has already started, can i still enroll?"

In [67]:
query

'the course has already started, can i still enroll?'

In [65]:
index.fit(documents)

In [66]:
boost = {"question": 3.0, "section": 0.5}

#Potenciamos unos parámetros respecto de otros, utilizando elasticsearch

results = index.search(
    query=query,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [68]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [ ]:
from openai import OpenAI
client = OpenAI()

In [70]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{
        'role': 'user', "content": query
    }]
)

In [71]:
response.choices[0].message.content

"It depends on the specific course and the policies of the institution offering it. Many educational institutions have add/drop periods during which you can still enroll in a course even after it has started, but this period is usually limited to the first few days or weeks of the term.\n\nHere are some steps you can take to find out if you can still enroll:\n\n1. **Check the Course Website:** Look for information about enrollment deadlines and policies related to late registration.\n   \n2. **Contact the Instructor:** Sometimes, instructors have the discretion to allow additional students to enroll after the official start date. Send an email or stop by during office hours to discuss your situation.\n\n3. **Visit the Registrar's Office:** They can provide the official enrollment deadlines and may offer options for late registration.\n\n4. **Talk to an Academic Advisor:** Advisors often have insight into the flexibility of registration policies and can guide you through the process.\n\

In [72]:
prompt_template = """
You are a teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering. If the answer is not in the context, output NONE. \n 
QUESTION: {question}
CONTEXT: {context}

"""

In [73]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']} \n question: {doc['question']} \n answer: {doc['text']}\n \n"

In [74]:
context

"section: General course-related questions \n question: Course - Can I still join the course after the start date? \n answer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n \nsection: General course-related questions \n question: Course - Can I follow the course after it finishes? \n answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n \nsection: General course-related questions \n question: Course - When will the course start? \n answer: The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h0

In [75]:
prompt = prompt_template.format(question=query,context=context).strip()

In [76]:
prompt

"You are a teaching assistant. Answer the QUESTION based on the CONTEXT. \nUse only the facts from the CONTEXT when answering. If the answer is not in the context, output NONE. \n \nQUESTION: the course has already started, can i still enroll?\nCONTEXT: section: General course-related questions \n question: Course - Can I still join the course after the start date? \n answer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n \nsection: General course-related questions \n question: Course - Can I follow the course after it finishes? \n answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n

In [79]:
raging = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{
        'role': 'user', "content": prompt
    }]
)

In [80]:
raging.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [83]:
def search(user_query):
    boost = {"question": 3.0, "section": 0.5}
    results = index.search(
        query=user_query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    ) 
    return results

In [85]:
results = search('how do i run kafka?')

In [86]:
def prompt_builder(user_query, search_results):
    prompt_template = """
    You are a teaching assistant. Answer the QUESTION based on the CONTEXT. 
    Use only the facts from the CONTEXT when answering. If the answer is not in the context, output NONE. \n 
    QUESTION: {question}
    CONTEXT: {context}
    
    """
    context = ""
    for doc in results:
        context = context + f"section: {doc['section']} \n question: {doc['question']} \n answer: {doc['text']}\n \n"
    prompt = prompt_template.format(question=query,context=context).strip()   
    return prompt

In [90]:
final_prompt = prompt_builder("how do i run kafka?", results)

In [93]:
def llm(prompt): 
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{
        'role': 'user', "content": prompt
    }]
    )
    return response.choices[0].message.content

In [94]:
llm(final_prompt)

'NONE'